In [5]:
pip install playsound==1.2.2

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install gtts

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyttsx3

     -------------------------------------- 184.3/184.3 kB 3.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


------

In [5]:
import threading
import cv2
import dlib
import pyttsx3

# 사운드로 변환
def speak():
    engine = pyttsx3.init()
    engine.say('좋조졸음 쉼터로 가세요 ')
    engine.runAndWait()

def main():
    face_detector = dlib.get_frontal_face_detector()
    eye_detector = dlib.shape_predictor('C:/face_detection/shape_predictor_68_face_landmarks.dat')
    # 눈 감은 상태 확인을 위한 변수 초기화
    # thread = threading.Thread(target=speak)
    webcam = cv2.VideoCapture(0)
    i = 0 
    while webcam.isOpened():
        status, frame = webcam.read()
        if not status:
            print('Could not read frame')
            break

        # 이미지를 그레이스케일로 변환
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 얼굴 감지
        faces = face_detector(gray)

        # 얼굴 영역에서 눈 찾기
        text = None  # 얼굴에 대한 텍스트 초기화

        for face in faces:
            landmarks = eye_detector(gray, face)
            # 얼굴의 위치 추출
            (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())

            # 눈 감음 여부 확인
            left_eye = landmarks.part(36).y - landmarks.part(37).y
            # right_eye = landmarks.part(45).y - landmarks.part(46).y
          
            if left_eye < 3:  # 눈을 감은 상태로 판단
                text = True
            else:
                text = False

            # 얼굴에 대한 텍스트 출력
            if text is True:
                cv2.putText(frame, 'sleep', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                i += 2
                if i >= 75:
                    thread = threading.Thread(target=speak)  # Create a new thread for each execution
                    thread.start()
                    i = 0
            elif text is False:
                cv2.putText(frame, 'awake', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                if i >= 0:
                    i -= 1
            else:
                if i >= 0:
                    i -= 2

        # 화면에 출력
        cv2.imshow('Drowsiness Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 웹캠과 창 닫기
    webcam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()